In [19]:
import finlab
from finlab import data
import pandas as pd
import datetime as dt
import numpy as np
import math
from dotenv import load_dotenv
import os
import time
def config():
    load_dotenv()


    
config()

finlab.login(str(os.getenv('FINLAB_API')))

輸入成功!


In [20]:
import pandas as pd

def get_feature_data(feature_name):
    """
    Helper function to get data for a single feature and return it as a DataFrame.
    """
    df = data.get(f'fundamental_features:{feature_name}').deadline()
    # Assuming the data has 'date' as one of the columns
    # df['date'] = pd.to_datetime(df['date'])
    # df.set_index('date', inplace=True)
    return df

def combine_feature_data(feature_list):
    """
    Main function to combine data from multiple features into a single DataFrame.
    """
    # List to hold dataframes
    dfs = []
    request_times = 0
    # Loop through each feature, get the data and append to the list
    for feature in feature_list:
        if request_times == 5:
            time.sleep(10)
        df = get_feature_data(feature)
        # Rename the column to match the feature for clarity in the combined DataFrame
        df.dropna(inplace=True, axis=1, how='all')
        dfs.append(df)

        # stack_df = df.stack(dropna=False)
        # dfs.append(stack_df)
        

        #cannot request the data too often
        time.sleep(1)
        request_times+=1

    
    # Combine all the dataframes on the index (date)
    combined_df = pd.concat(dfs, axis=1, keys=feature_list)
    return combined_df.stack(dropna=False)

# List of all the features to retrieve and combine
feature_list = ['EBITDA', '營運現金流', '歸屬母公司淨利', '折舊', '流動資產', '流動負債', '取得不動產廠房及設備',
    '經常稅後淨利', 'ROA稅後息前', 'ROA綜合損益', 'ROE稅後', 'ROE綜合損益', '稅前息前折舊前淨利率', '營業毛利率',
    '營業利益率', '稅前淨利率', '稅後淨利率', '業外收支營收率', '貝里比率', '營業費用率', '推銷費用率', '管理費用率',
    '研究發展費用率', '現金流量比率', '稅率', '每股營業額', '每股營業利益', '每股現金流量', '每股稅前淨利', '每股綜合損益',
    '每股稅後淨利', '總負債除總淨值', '負債比率', '淨值除資產', '營收成長率', '營業毛利成長率', '營業利益成長率', '稅前淨利成長率',
    '稅後淨利成長率', '資產總額成長率', '淨值成長率', '流動比率', '速動比率', '利息支出率', '營運資金', '總資產週轉次數', '應收帳款週轉率'
]

# Placeholder for the actual data source
data_source = {}  # This would be the actual data source in practice

# Call the function to combine the data
combined_feature_df = combine_feature_data(feature_list)

# Assuming you would like to see the combined dataframe
combined_feature_df  # Display the combined dataframe
price_data = data.get('etl:adj_close')

Daily usage: 60.0 / 500 MB - fundamental_features:負債比率
Daily usage: 61.3 / 500 MB - fundamental_features:淨值除資產
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/max/Library/Python/3.11/lib/python/site-packages/urllib3/connectionpool.py", line 703, in urlopen
  File "/Users/max/Library/Python/3.11/lib/python/site-packages/urllib3/connectionpool.py", line 386, in _make_request
  File "/Users/max/Library/Python/3.11/lib/python/site-packages/urllib3/connectionpool.py", line 1042, in _validate_conn
  File "/Users/max/Library/Python/3.11/lib/python/site-packages/urllib3/connection.py", line 419, in connect
  File "/Users/max/Library/Python/3.11/lib/python/site-packages/urllib3/util/ssl_.py", line 449, in ssl_wrap_socket
  File "/Users/max/Library/Python/3.11/lib/python/site-packages/urllib3/util/ssl_.py", line 493, in _ssl_wrap_socket_impl
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/ssl.py", line 517, in wrap_socket
    return self.sslsocket_class._create(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.1

In [ ]:
import pandas as pd


def calculate_returns(df, date_index):
    # Convert the first level of the multi-index to a list
    # first_level_values = df.index.get_level_values(0).unique()
    first_level_values_list = date_index.tolist()

    # Sort the list of dates (if they are not in chronological order)
    first_level_values_list.sort()

    # Initialize a DataFrame to store returns
    quarterly_returns = pd.DataFrame(index=first_level_values_list, columns=df.columns)

    # Calculate returns for each 'quarter' in the first_level_values_list
    for i in range(len(first_level_values_list) - 1):
        start = first_level_values_list[i]
        end = first_level_values_list[i + 1]

        # Get start and end prices for the 'quarter'
        start_prices = df.loc[df.index.asof(start)]
        end_prices = df.loc[df.index.asof(end)]

        # Calculate returns
        returns = (end_prices - start_prices) / start_prices * 100
        
        # Assign returns to the DataFrame
        quarterly_returns.loc[end] = returns

    return quarterly_returns

# Load your DataFrame here
# df = pd.read_csv('your_data.csv')
# Assuming 'date' is part of the multi-index and each stock is a column
# df.index = pd.to_datetime(df.index)  # Convert the date index to datetime if necessary

# Call the function with your DataFrame
# returns_df = calculate_returns(df)
# Process or analyze the returns DataFrame as needed
y = calculate_returns(price_data, date_index.drop_duplicates())
y = y.stack()
y.name = 'return'

In [ ]:

dataset = pd.concat([cleaned_combined_feature_df,y],axis=1)
dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset.dropna(axis=0, how='any', inplace=True)
dataset = dataset.astype(float)



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))  # Output layer; change activation based on the problem

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")


predictions = model.predict(X_test)


In [2]:
# 營業利益 "data.get('fundamental_features:營業利益')""	float	
# EBITDA	"data.get('fundamental_features:EBITDA')"	float	
# 營運現金流	"data.get('fundamental_features:營運現金流')"	float	
# 歸屬母公司淨利	"data.get('fundamental_features:歸屬母公司淨利')	float	
# 折舊	data.get('fundamental_features:折舊')	float	
# 流動資產	data.get('fundamental_features:流動資產')	float	
# 流動負債	data.get('fundamental_features:流動負債')	float	
# 取得不動產廠房及設備	data.get('fundamental_features:取得不動產廠房及設備')
# 經常稅後淨利	data.get('fundamental_features:經常稅後淨利')	float	
# ROA稅後息前	data.get('fundamental_features:ROA稅後息前')	float	
# ROA綜合損益	data.get('fundamental_features:ROA綜合損益')	float	
# ROE稅後	data.get('fundamental_features:ROE稅後')	float	
# ROE綜合損益	data.get('fundamental_features:ROE綜合損益')	float	
# 稅前息前折舊前淨利率	data.get('fundamental_features:稅前息前折舊前淨利率')	float	
# 營業毛利率	data.get('fundamental_features:營業毛利率')	float	
# 營業利益率	data.get('fundamental_features:營業利益率')	float
# 稅前淨利率	data.get('fundamental_features:稅前淨利率')	float	
# 稅後淨利率	data.get('fundamental_features:稅後淨利率')	float	
# 業外收支營收率	data.get('fundamental_features:業外收支營收率')	float	
# 貝里比率	data.get('fundamental_features:貝里比率')	float	
# 營業費用率	data.get('fundamental_features:營業費用率')	float	
# 推銷費用率	data.get('fundamental_features:推銷費用率')	float	
# 管理費用率	data.get('fundamental_features:管理費用率')	float	
# 研究發展費用率	data.get('fundamental_features:研究發展費用率')	float	
# 現金流量比率	data.get('fundamental_features:現金流量比率')	float	
# 稅率	data.get('fundamental_features:稅率')	float	
# 每股營業額	data.get('fundamental_features:每股營業額')	float	
# 每股營業利益	data.get('fundamental_features:每股營業利益')	float	
# 每股現金流量	data.get('fundamental_features:每股現金流量')	float	
# 每股稅前淨利	data.get('fundamental_features:每股稅前淨利')	float	
# 每股綜合損益	data.get('fundamental_features:每股綜合損益')	float	
# 每股稅後淨利	data.get('fundamental_features:每股稅後淨利')	float
# 總負債除總淨值	data.get('fundamental_features:總負債除總淨值')	float	
# 負債比率	data.get('fundamental_features:負債比率')	float	
# 淨值除資產	data.get('fundamental_features:淨值除資產')	float	
# 營收成長率	data.get('fundamental_features:營收成長率')	float	
# 營業毛利成長率	data.get('fundamental_features:營業毛利成長率')	float	
# 營業利益成長率	data.get('fundamental_features:營業利益成長率')	float	
# 稅前淨利成長率	data.get('fundamental_features:稅前淨利成長率')	float	
# 稅後淨利成長率	data.get('fundamental_features:稅後淨利成長率')	float	
# 資產總額成長率	data.get('fundamental_features:資產總額成長率')	float	
# 淨值成長率	data.get('fundamental_features:淨值成長率')	float	
# 流動比率	data.get('fundamental_features:流動比率')	float	
# 速動比率	data.get('fundamental_features:速動比率')	float	
# 利息支出率	data.get('fundamental_features:利息支出率')	float	
# 營運資金	data.get('fundamental_features:營運資金')	float	
# 總資產週轉次數	data.get('fundamental_features:總資產週轉次數')	float	
# 應收帳款週轉率	data.get('fundamental_features:應收帳款週轉率')	float	



In [45]:
roe = data.get('fundamental_features:ROE稅後').deadline()
Receivables_turnover_ratio = data.get('fundamental_features:應收帳款週轉率').deadline()
profit = data.get('fundamental_features:營業利益').deadline()
profit.dropna(axis=1, how='all').stack(dropna=False)
price_data = data.get('etl:adj_close')

In [105]:
Receivables_turnover_ratio[Receivables_turnover_ratio.columns.isin(price_data_stock_columns)]

ValueError: Item wrong length 1781 instead of 55.

In [18]:
Receivables_turnover_ratio.stack()

date     symbol
2013-Q2  1101       1.499238
         1102       1.170351
         1103       1.895246
         1104       1.279083
         1108       0.525799
                     ...    
2020-Q3  9951       1.376963
         9955      50.620248
         9958       2.555440
         9960       8.651311
         9962       6.943470
Length: 47140, dtype: float64

In [7]:
cleaned_roe = roe.dropna(axis=1)

In [27]:
cleaned_roe.reset_index()
# turn it into a multi index dataframe
stack_roe = cleaned_roe.stack(dropna=False)
stack_Receivables_turnover_ratio = Receivables_turnover_ratio.stack(dropna=False)
a = pd.concat([cleaned_roe, Receivables_turnover_ratio], axis=1)
a.stack()

date     symbol
2013-Q1  1101       1.375582
         1102       1.315619
         1103       0.363729
         1104       1.335893
         1108       1.491161
                     ...    
2020-Q3  9951       1.376963
         9955      50.620248
         9958       2.555440
         9960       8.651311
         9962       6.943470
Length: 85921, dtype: float64

In [13]:
import pandas as pd

def get_feature_data(feature_name):
    """
    Helper function to get data for a single feature and return it as a DataFrame.
    """
    df = data.get(f'fundamental_features:{feature_name}').deadline()
    # Assuming the data has 'date' as one of the columns
    # df['date'] = pd.to_datetime(df['date'])
    # df.set_index('date', inplace=True)
    return df

def combine_feature_data(feature_list):
    """
    Main function to combine data from multiple features into a single DataFrame.
    """
    # List to hold dataframes
    dfs = []
    request_times = 0
    # Loop through each feature, get the data and append to the list
    for feature in feature_list:
        if request_times == 5:
            time.sleep(5)
        df = get_feature_data(feature)
        # Rename the column to match the feature for clarity in the combined DataFrame
        df.dropna(inplace=True, axis=1, how='all')
        dfs.append(df)

        # stack_df = df.stack(dropna=False)
        # dfs.append(stack_df)
        

        #cannot request the data too often
        time.sleep(1)
        request_times+=1

    
    # Combine all the dataframes on the index (date)
    combined_df = pd.concat(dfs, axis=1, keys=feature_list)
    return combined_df.stack(dropna=False)

# List of all the features to retrieve and combine
feature_list = ['EBITDA', '營運現金流', '歸屬母公司淨利', '折舊', '流動資產', '流動負債', '取得不動產廠房及設備',
    '經常稅後淨利', 'ROA稅後息前', 'ROA綜合損益', 'ROE稅後', 'ROE綜合損益', '稅前息前折舊前淨利率', '營業毛利率',
    '營業利益率', '稅前淨利率', '稅後淨利率', '業外收支營收率', '貝里比率', '營業費用率', '推銷費用率', '管理費用率',
    '研究發展費用率', '現金流量比率', '稅率', '每股營業額', '每股營業利益', '每股現金流量', '每股稅前淨利', '每股綜合損益',
    '每股稅後淨利', '總負債除總淨值', '負債比率', '淨值除資產', '營收成長率', '營業毛利成長率', '營業利益成長率', '稅前淨利成長率',
    '稅後淨利成長率', '資產總額成長率', '淨值成長率', '流動比率', '速動比率', '利息支出率', '營運資金', '總資產週轉次數', '應收帳款週轉率'
]


# Call the function to combine the data
combined_feature_df = combine_feature_data(feature_list)

# Assuming you would like to see the combined dataframe
combined_feature_df  # Display the combined dataframe
price_data = data.get('etl:adj_close')

ConnectionError: ('Connection aborted.', OSError(22, 'Invalid argument'))

In [ ]:
cleaned_combined_feature_df= combined_feature_df.dropna(axis=0, how='all')
date_index = cleaned_combined_feature_df.index.get_level_values(0).drop_duplicates()
date_index
stock_index = cleaned_combined_feature_df.index.get_level_values(1).drop_duplicates()
stock_index

Index(['2801', '2809', '2812', '2816', '2820', '2832', '2834', '2836', '2838',
       '2845',
       ...
       '6748', '6749', '6753', '6775', '6778', '6779', '7561', '7566', '6029',
       '6782'],
      dtype='object', name='symbol', length=2404)

In [6]:
import pandas as pd


def calculate_returns(df, date_index):
    # Convert the first level of the multi-index to a list
    # first_level_values = df.index.get_level_values(0).unique()
    first_level_values_list = date_index.tolist()

    # Sort the list of dates (if they are not in chronological order)
    first_level_values_list.sort()

    # Initialize a DataFrame to store returns
    quarterly_returns = pd.DataFrame(index=first_level_values_list, columns=df.columns)

    # Calculate returns for each 'quarter' in the first_level_values_list
    for i in range(len(first_level_values_list) - 1):
        start = first_level_values_list[i]
        end = first_level_values_list[i + 1]

        # Get start and end prices for the 'quarter'
        start_prices = df.loc[df.index.asof(start)]
        end_prices = df.loc[df.index.asof(end)]

        # Calculate returns
        returns = (end_prices - start_prices) / start_prices * 100
        
        # Assign returns to the DataFrame
        quarterly_returns.loc[end] = returns

    return quarterly_returns

# Load your DataFrame here
# df = pd.read_csv('your_data.csv')
# Assuming 'date' is part of the multi-index and each stock is a column
# df.index = pd.to_datetime(df.index)  # Convert the date index to datetime if necessary

# Call the function with your DataFrame
# returns_df = calculate_returns(df)
# Process or analyze the returns DataFrame as needed
y = calculate_returns(price_data, date_index.drop_duplicates())
y = y.stack()
y.name = 'return'

NameError: name 'price_data' is not defined

In [ ]:
# price_data.loc[price_data.index.asof('2013-01-02')]
# # Get start and end prices for the 'quarter'
# start_prices = price_data.loc[price_data.index.asof('2013-01-02')]
# end_prices = price_data.loc[price_data.index.asof('2013-01-03')]

# # Calculate returns
# returns = (end_prices - start_prices) / start_prices * 100
# quarterly_returns = pd.DataFrame(index=date_index, columns=price_data.columns)
# date_index.drop_duplicates()

In [97]:

dataset = pd.concat([cleaned_combined_feature_df,y],axis=1)
dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset.dropna(axis=0, how='any', inplace=True)
dataset = dataset.astype(float)



In [116]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()




# Splitting the data into features (X) and target (y)
X = dataset.drop('return', axis=1)
y = dataset['return'].values.reshape(-1,1)
print(X)

scaled_X = scaler.fit_transform(X)
scaled_y = scaler.fit_transform(y)

# Example: Log transformation for positive values

# Splitting the dataset into training and prediction sets
# Assuming that the rows with NaN in 'return' are for prediction
X_train, X_temp, y_train, y_temp = train_test_split(scaled_X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_val
# X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
# X_train.dropna()

# X_train_scaled = scaler.fit_transform(X_train.replace([np.inf, -np.inf], np.nan))
# X_train_scaled






                      EBITDA      營運現金流    歸屬母公司淨利         折舊        流動資產   
           symbol                                                           
2014-05-15 1101    5708941.0  7756626.0  1833139.0  1594505.0  92121458.0  \
           1104     288321.0    73268.0   239661.0    32471.0   2536190.0   
           1201     676228.0   -93950.0   256386.0   287376.0  22487149.0   
           1203     234424.0    16778.0    85357.0    43224.0   3081351.0   
           1210     912770.0   164056.0   356947.0   368409.0  20971573.0   
...                      ...        ...        ...        ...         ...   
2020-11-16 9950      -8805.0   -47033.0   -72973.0    50734.0    895866.0   
           9951     222220.0   228275.0   147858.0    36283.0   3282463.0   
           9955      37273.0   -82356.0    20674.0    12910.0    937019.0   
           9958     601440.0   474341.0   285666.0    73565.0   6660614.0   
           9962       9782.0   147290.0     3727.0     4674.0    740617.0   

array([[ 1.06398515e-01,  1.49241588e-01,  2.51575145e-01, ...,
         4.63296956e-01,  4.18490826e-01, -3.24880356e-02],
       [-5.89192999e-02, -8.07251506e-03, -4.80735603e-02, ...,
         2.53676612e-02, -9.62214365e-01, -4.09844869e-02],
       [-1.22524679e-01, -2.62316797e-02, -1.13343307e-01, ...,
        -9.22664948e-02, -8.19492830e-01,  7.47872601e-02],
       ...,
       [ 2.22457043e-02, -9.39706905e-02,  8.96797938e-02, ...,
        -1.42928589e-01, -1.38059302e+00, -4.82584302e-02],
       [ 6.48656256e-01,  4.48232444e-01,  1.03993983e+00, ...,
         2.31819670e-01,  1.39327314e-01, -5.37407982e-04],
       [-7.80869484e-02, -5.82646353e-02, -6.48689774e-02, ...,
        -1.29126996e-01, -4.51898592e-01, -5.45782357e-02]])

In [115]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

baseline_value = y_train.mean()
# Or use the median: baseline_value = y_train.median()

baseline_predictions = [baseline_value] * len(y_test)


baseline_mse = mean_squared_error(y_test, baseline_predictions)
print(f"Baseline Model Mean Squared Error: {baseline_mse}")


# Initialize the Random Forest Regressor
random_forest_model = RandomForestRegressor(n_estimators=100, random_state=42)


# Fit the model on the training data
random_forest_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = random_forest_model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Baseline Model Mean Squared Error: 301.87060133713186


/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Mean Squared Error: 357.25407092103086


In [109]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))  # Output layer; change activation based on the problem

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")


predictions = model.predict(X_test)


Epoch 1/100
970/970 [==============================] - 1s 889us/step - loss: 0.9715 - mae: 0.5826 - val_loss: 1.0074 - val_mae: 0.6025
Epoch 2/100
970/970 [==============================] - 1s 834us/step - loss: 0.9591 - mae: 0.5802 - val_loss: 0.9983 - val_mae: 0.5970
Epoch 3/100
970/970 [==============================] - 1s 798us/step - loss: 0.9563 - mae: 0.5801 - val_loss: 1.0065 - val_mae: 0.5953
Epoch 4/100
970/970 [==============================] - 1s 799us/step - loss: 0.9552 - mae: 0.5788 - val_loss: 1.0083 - val_mae: 0.6078
Epoch 5/100
970/970 [==============================] - 1s 796us/step - loss: 0.9514 - mae: 0.5790 - val_loss: 0.9972 - val_mae: 0.6033
Epoch 6/100
970/970 [==============================] - 1s 833us/step - loss: 0.9512 - mae: 0.5787 - val_loss: 0.9979 - val_mae: 0.6038
Epoch 7/100
970/970 [==============================] - 1s 796us/step - loss: 0.9489 - mae: 0.5784 - val_loss: 1.0020 - val_mae: 0.6001
Epoch 8/100
970/970 [==============================] - 

In [1]:
y_pred.max()

NameError: name 'y_pred' is not defined

In [110]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(1, activation='linear'))  # Output layer; change activation based on the problem

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")


predictions = model.predict(X_test)


ValueError: Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 128)